In [1]:
import streamlit as st
import sys
import os

# 프로젝트 루트 디렉토리를 Python path에 추가
project_root = os.path.dirname(os.path.dirname(os.path.abspath('.')))
if project_root not in sys.path:
    sys.path.append(project_root)

from app.orchestrator.prompts.prompt import ORCHESTRATOR_AGENT_PROMPT
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [2]:
# Streamlit 페이지 설정
st.set_page_config(
    page_title="의료 AI 상담 서비스",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# 사이드바 설정
with st.sidebar:
    st.title("🏥 의료 AI 상담")
    st.markdown("---")
    st.markdown("### 상담 진행 단계")
    
    # 세션 상태 초기화
    if "conversation_stage" not in st.session_state:
        st.session_state.conversation_stage = "greeting"
    if "collected_info" not in st.session_state:
        st.session_state.collected_info = {}
    if "conversation_history" not in st.session_state:
        st.session_state.conversation_history = []
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # 진행 상태 표시
    stages = {
        "greeting": "🤝 인사 및 안내",
        "basic_info": "📝 기본 정보 수집",
        # "symptoms": "🩺 증상 확인",
        # "history": "📋 병력 조사",
        # "lifestyle": "🏃‍♂️ 생활습관 확인",
        "image_upload": "📷 이미지 업로드",
        "analysis": "🔍 분석 중",
        "completed": "✅ 상담 완료"
    }
    
    for stage, description in stages.items():
        if stage == st.session_state.conversation_stage:
            st.markdown(f"**➤ {description}**")
        else:
            st.markdown(f"   {description}")
    
    st.markdown("---")
    if st.button("대화 초기화"):
        for key in ["conversation_stage", "collected_info", "conversation_history", "messages"]:
            if key in st.session_state:
                del st.session_state[key]
        st.rerun()

2025-08-27 11:50:02.714 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:02.715 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:02.757 
  command:

    streamlit run /opt/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-08-27 11:50:02.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:02.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:02.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:02.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running 

In [3]:
# 오케스트레이터 에이전트 클래스
class MedicalOrchestrator:
    def __init__(self):
        # 실제 환경에서는 환경변수에서 API 키를 가져오세요
        # self.llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
        pass
    
    def process_conversation(self, user_input, conversation_stage, collected_info, conversation_history, has_image=False):
        """
        사용자 입력을 처리하고 응답을 생성합니다.
        실제 환경에서는 여기서 LangChain을 사용하여 LLM을 호출합니다.
        """
        # 데모용 응답 로직 (실제로는 ORCHESTRATOR_AGENT_PROMPT를 사용)
        if conversation_stage == "greeting":
            return {
                "decision": "CONTINUE_CONVERSATION",
                "message": "안녕하세요! 건강검진 상담을 도와드리는 의료 AI입니다. 😊\\n\\n먼저 기본적인 정보를 여쭤보겠습니다. 성함과 나이를 알려주시겠어요?",
                "next_stage": "basic_info",
                "collected_info": collected_info
            }
        elif conversation_stage == "basic_info":
            # 기본 정보 수집
            collected_info["basic_response"] = user_input
            return {
                "decision": "CONTINUE_CONVERSATION", 
                "message": f"감사합니다! 현재 어떤 증상이나 불편한 점이 있으신지 자세히 말씀해 주세요.",
                "next_stage": "symptoms",
                "collected_info": collected_info
            }
        elif conversation_stage == "symptoms":
            collected_info["symptoms"] = user_input
            return {
                "decision": "CONTINUE_CONVERSATION",
                "message": "증상에 대해 말씀해 주셔서 감사합니다. 과거에 큰 병을 앓으신 적이 있거나 수술을 받으신 경험이 있나요?",
                "next_stage": "history", 
                "collected_info": collected_info
            }
        elif conversation_stage == "history":
            collected_info["medical_history"] = user_input
            return {
                "decision": "CONTINUE_CONVERSATION",
                "message": "네, 알겠습니다. 평소 흡연이나 음주는 하시는지, 그리고 운동은 얼마나 자주 하시는지 알려주세요.",
                "next_stage": "lifestyle",
                "collected_info": collected_info
            }
        elif conversation_stage == "lifestyle":
            collected_info["lifestyle"] = user_input
            return {
                "decision": "REQUEST_IMAGE",
                "message": "생활습관에 대해 잘 알겠습니다. 이제 X-ray 이미지가 있으시면 업로드해 주세요. 더 정확한 분석을 도와드릴 수 있습니다.",
                "next_stage": "image_upload",
                "collected_info": collected_info
            }
        else:
            return {
                "decision": "END_CONSULTATION",
                "message": "상담이 완료되었습니다. 궁금한 점이 더 있으시면 언제든 문의해 주세요!",
                "next_stage": "completed",
                "collected_info": collected_info
            }

# 오케스트레이터 인스턴스 생성
orchestrator = MedicalOrchestrator()

In [4]:
# 메인 인터페이스
st.title("🏥 의료 AI 상담 서비스")
st.markdown("건강검진을 위한 AI 상담을 시작합니다. 편안하게 대화해 주세요.")

# 채팅 히스토리 표시
chat_container = st.container()

with chat_container:
    # 초기 메시지 표시
    if not st.session_state.messages:
        initial_response = orchestrator.process_conversation(
            "", "greeting", st.session_state.collected_info, 
            st.session_state.conversation_history
        )
        st.session_state.messages.append({"role": "assistant", "content": initial_response["message"]})
        st.session_state.conversation_stage = initial_response["next_stage"]
    
    # 메시지 히스토리 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# 사용자 입력
if prompt := st.chat_input("메시지를 입력하세요..."):
    # 사용자 메시지 추가
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.session_state.conversation_history.append(f"사용자: {prompt}")
    
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # AI 응답 생성
    with st.chat_message("assistant"):
        with st.spinner("답변을 생성하는 중..."):
            response = orchestrator.process_conversation(
                prompt,
                st.session_state.conversation_stage,
                st.session_state.collected_info,
                st.session_state.conversation_history
            )
            
            # 응답 표시
            st.markdown(response["message"])
            
            # 세션 상태 업데이트
            st.session_state.messages.append({"role": "assistant", "content": response["message"]})
            st.session_state.conversation_stage = response["next_stage"]
            st.session_state.collected_info = response["collected_info"]
            st.session_state.conversation_history.append(f"AI: {response['message']}")

# 이미지 업로드 섹션 (image_upload 단계에서만 표시)
if st.session_state.conversation_stage == "image_upload":
    st.markdown("---")
    st.subheader("📷 X-ray 이미지 업로드")
    
    uploaded_file = st.file_uploader(
        "X-ray 이미지를 업로드하세요",
        type=['png', 'jpg', 'jpeg', 'dcm'],
        help="PNG, JPG, JPEG, DICOM 형식을 지원합니다."
    )
    
    if uploaded_file is not None:
        # 이미지 표시
        col1, col2 = st.columns([1, 1])
        
        with col1:
            st.image(uploaded_file, caption="업로드된 X-ray 이미지", use_column_width=True)
        
        with col2:
            st.info("이미지가 업로드되었습니다!")
            if st.button("이미지 분석 시작", type="primary"):
                st.session_state.conversation_stage = "analysis"
                st.session_state.messages.append({
                    "role": "assistant", 
                    "content": "이미지 분석을 시작합니다. 전문 AI가 X-ray 이미지를 분석하고 있습니다..."
                })
                st.rerun()

# 수집된 정보 표시 (디버깅용)
if st.session_state.collected_info:
    with st.expander("수집된 정보 확인"):
        st.json(st.session_state.collected_info)

2025-08-27 11:50:06.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.812 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.813 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 11:50:06.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## Streamlit 실행 방법

위의 코드를 실행하려면 다음 명령어를 사용하세요:

```bash
streamlit run notebooks/first_service.ipynb
```

또는 Python 스크립트로 변환 후 실행:

```bash
jupyter nbconvert --to script notebooks/first_service.ipynb
streamlit run notebooks/first_service.py
```

## 주요 기능

1. **단계별 상담 진행**: 사이드바에서 현재 진행 단계를 확인할 수 있습니다
2. **채팅 인터페이스**: 자연스러운 대화형 상담
3. **이미지 업로드**: X-ray 이미지 업로드 및 분석 요청
4. **세션 관리**: 대화 히스토리와 수집된 정보를 세션에 저장
5. **정보 수집**: 기본 정보, 증상, 병력, 생활습관 등을 체계적으로 수집

## 실제 배포 시 추가할 사항

- OpenAI API 키 설정 및 LangChain 연동
- 실제 의료 이미지 분석 모델 연동 (medblip 에이전트)
- 데이터베이스 연결 및 상담 기록 저장
- 보안 및 개인정보 보호 강화